# 新版模型

In [5]:

import pandas as pd
import numpy as np
import pulp
import time
from datetime import datetime
import matplotlib.pyplot as plt

## Model setting

In [2]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()


load c:\Users\24081\AppData\Local\Programs\Python\Python310\lib\site-packages\ortools\.libs\zlib1.dll...
load c:\Users\24081\AppData\Local\Programs\Python\Python310\lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\Users\24081\AppData\Local\Programs\Python\Python310\lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\Users\24081\AppData\Local\Programs\Python\Python310\lib\site-packages\ortools\.libs\re2.dll...
load c:\Users\24081\AppData\Local\Programs\Python\Python310\lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\Users\24081\AppData\Local\Programs\Python\Python310\lib\site-packages\ortools\.libs\highs.dll...
load c:\Users\24081\AppData\Local\Programs\Python\Python310\lib\site-packages\ortools\.libs\ortools.dll...


In [6]:
# Decision Variables

X = {}  # X[i, j, k, t] = 1 表示课程 i 在第 j 周时间 t 被安排在教室 k
for i in I:      # 遍历所有课程
    for j in J:  # 遍历所有周
        for k in K:  # 遍历所有教室
            for t in T:  # 遍历所有时间段
                X[i, j, k, t] = model.NewBoolVar(f'X_{i}_{j}_{k}_{t}')


NameError: name 'I' is not defined

In [ ]:
# Constraints1：同一时间、同一教室只能安排一门课
for j in J:
    for k in K:
        for t in T:
            model.Add(sum(X[i, j, k, t] for i in I) <= 1)


In [ ]:
# Constraints2：每门课程不能安排到多个教室
for i in I:
    for j in J:
        for t in T:
            model.Add(sum(X[i, j, k, t] for k in K) <= 1)


In [ ]:
# Constraints3：课程人数不能超过教室容量
for i in I:
    for j in J:
        for k in K:
            for t in T:
                model.Add(X[i, j, k, t] * E[i] <= S[k])
                # E[i] is the number of students enrolled in course i.
                # S[k] is the maximum capacity of classroom k.


In [ ]:
# Objective Function
# Minimize violations of soft constraints.

v1 = model.NewIntVar(0, 10000, "room_utilization_violation")  # 教室利用率惩罚
v2 = model.NewIntVar(0, 10000, "student_conflict_violation")  # 学生冲突惩罚
v3 = model.NewIntVar(0, 10000, "schedule_variance_violation")  # 课程时间分布不均衡惩罚

In [ ]:
# Soft Constraints

# Calculate waste in classroom use
model.Add(v1 == sum((1 - (E[i] / S[k])) * X[i, j, k, t] for i in I for j in J for k in K for t in T))



# The number of times a student takes multiple classes at the same time
model.Add(v2 == sum((sum(X[i, j, k, t] * Y[s, i] for i in I) > 1) for s in S for j in J for k in K for t in T))
                                    # Y[s, i] Whether student s takes the course i (0/1)


#  Calculate whether the course schedule is evenly spaced
average_schedule = sum(X[i, j, k, t] for i in I for j in J for k in K for t in T) / (37 * 5 * 18)
model.Add(v3 == sum((sum(X[i, j, k, t] for i in I for k in K for t in T) - average_schedule) ** 2 for j in J))


In [ ]:

w1, w2, w3 = 1, 1, 1  # soft constraint weights
model.Minimize(w1 * v1 + w2 * v2 + w3 * v3)
